In [1]:
#Importing the required libraries
import pandas as pd
from datetime import datetime, timedelta, date
import numpy as np

In [48]:
#Importing the required filtered dataset
df2=pd.read_csv("CSV/115_filtered2.csv")

In [49]:
#Extract the sensor start time from the time converted date-time column
df2['Starttime'] = df2['Starttime'].apply(lambda x : datetime.strptime(x, '%H:%M:%S').time())

In [50]:
#Extract the sensor end time from the time converted date-time column
df2['Endtime'] = df2['Endtime'].apply(lambda x : datetime.strptime(x, '%H:%M:%S').time())

### Getting the Minute column

In [51]:
df2['Minutes'] = df2['Starttime'].map(lambda x : x.minute)

In [52]:
df2['End Minutes'] = df2['Endtime'].map(lambda x : x.minute)

### Finding the maximum duration for which a particular sensor activation lasted

In [53]:
(df2['End Minutes']-df2['Minutes']).max()

1

In [54]:
df2.drop('End Minutes', axis=1, inplace=True)

### Getting the Bits columns

In [55]:
def getbit1(x):
    if (x==0 or x==5 or x==10 or x==15 or x==20 or x==25 or x==30 or x==35 or x==40 or x==45 or x==50 or x==55):
        return 1
    else:
        return 0

In [56]:
def getbit2(x):
    if (x==1 or x==6 or x==11 or x==16 or x==21 or x==26 or x==31 or x==36 or x==41 or x==46 or x==51 or x==56):
        return 1
    else:
        return 0

In [57]:
def getbit3(x):
    if (x==2 or x==7 or x==12 or x==17 or x==22 or x==27 or x==32 or x==37 or x==42 or x==47 or x==52 or x==57):
        return 1
    else:
        return 0

In [58]:
def getbit4(x):
    if (x==3  or x==8 or x==13 or x==18 or x==23 or x==28 or x==33 or x==38 or x==43 or x==48 or x==53 or x==58):
        return 1
    else:
        return 0

In [59]:
def getbit5(x):
    if (x==4 or x==9 or x==14 or x==19 or x==24 or x==29 or x==34 or x==39 or x==44 or x==49 or x==54 or x==59):
        return 1
    else:
        return 0

In [60]:
df2['bit1']= np.array(df2['Minutes'].apply(getbit1))
df2['bit2']= np.array(df2['Minutes'].apply(getbit2))
df2['bit3']= np.array(df2['Minutes'].apply(getbit3))
df2['bit4']= np.array(df2['Minutes'].apply(getbit4))
df2['bit5']= np.array(df2['Minutes'].apply(getbit5))

In [90]:
df2.iloc[60:65]

,SensorID,Start,End,Placeholder,Date,Starttime,Endtime,Minutes,bit1,bit2,bit3,bit4,bit5
60,363,19-06-06 00:01:21,19-06-06 00:01:22,1.0,19-06-06,00:01:21,00:01:22,1,0,1,0,0,0
61,421,19-06-06 00:03:54,19-06-06 00:03:55,1.0,19-06-06,00:03:54,00:03:55,3,0,0,0,1,0
62,422,19-06-06 00:03:57,19-06-06 00:03:58,1.0,19-06-06,00:03:57,00:03:58,3,0,0,0,1,0
63,422,19-06-06 00:03:58,19-06-06 00:03:59,1.0,19-06-06,00:03:58,00:03:59,3,0,0,0,1,0
64,422,19-06-06 00:04:07,19-06-06 00:04:08,1.0,19-06-06,00:04:07,00:04:08,4,0,0,0,0,1


In [91]:
df2.tail()

,SensorID,Start,End,Placeholder,Date,Starttime,Endtime,Minutes,bit1,bit2,bit3,bit4,bit5
2511174,425,18-07-06 23:58:25,18-07-06 23:58:26,1.0,18-07-06,23:58:25,23:58:26,58,0,0,0,1,0
2511175,427,18-07-06 23:58:26,18-07-06 23:58:27,1.0,18-07-06,23:58:26,23:58:27,58,0,0,0,1,0
2511176,428,18-07-06 23:58:26,18-07-06 23:58:27,1.0,18-07-06,23:58:26,23:58:27,58,0,0,0,1,0
2511177,423,18-07-06 23:58:28,18-07-06 23:58:29,1.0,18-07-06,23:58:28,23:58:29,58,0,0,0,1,0
2511178,422,18-07-06 23:58:30,18-07-06 23:58:31,1.0,18-07-06,23:58:30,23:58:31,58,0,0,0,1,0


### A separate template of 5 minutes interval in a day is created

In [64]:
timeiter = pd.read_excel('timetemplate.xlsx')

In [65]:
timeiter.head()

,start,end
0,00:00:01,00:05:00
1,00:05:00,00:10:00
2,00:10:00,00:15:00
3,00:15:00,00:20:00
4,00:20:00,00:25:00


In [66]:
df7 = pd.DataFrame(data={'SensorID':[0], 'bit1':[0], 'bit2':[0], 'bit3':[0], 'bit4':[0], 'bit5':[0], 'Starttime':[0], 'Endtime':[0]})

### Grouping By Sensor ID in each 5 minute interval to get bits corresponding to that 5 minute interval

In [67]:
def wordgen(tinterval):
    global df7
    df6 = df5[(df5['Starttime']>=tinterval[0]) & (df5['Endtime']<=datetime.strptime(str(tinterval[1]), '%H:%M:%S').time())].groupby(by='SensorID').sum()[['bit1', 'bit2', 'bit3', 'bit4', 'bit5']].copy()
    df6.reset_index(inplace=True)
    for string in ['bit1','bit2','bit3','bit4','bit5']:
        df6[string] = df6[string].map(lambda x : 1 if x>1 else x)
    df6['Starttime'] = tinterval[0]
    df6['Endtime'] = tinterval[1]
    if(not(df6.empty)):
        df7=pd.concat([df7,df6])

### Getting the Segment column
* 00:00 to 06:00 -> Segment 1
* 06:00 to 07:00 -> Segment 2
* 07:00 to 09:00 -> Segment 3
* 09:00 to 11:00 -> Segment 4
* 11:00 to 14:00 -> Segment 5
* 14:00 to 17:00 -> Segment 6
* 17:00 to 19:00 -> Segment 7
* 19:00 to 21:00 -> Segment 8
* 21:00 to 00:00 -> Segment 9

In [88]:
def get_segment(x):
    if ((x >= datetime.strptime("00:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("06:00:00", '%H:%M:%S').time())):
        return '1'
    if ((x >= datetime.strptime("06:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("07:00:00", '%H:%M:%S').time())):
        return '2'
    if ((x >= datetime.strptime("07:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("09:00:00", '%H:%M:%S').time())):
        return '3'
    if ((x >= datetime.strptime("09:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("11:00:00", '%H:%M:%S').time())):
        return '4'
    if ((x >= datetime.strptime("11:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("14:00:00", '%H:%M:%S').time())):
        return '5'
    if ((x >= datetime.strptime("14:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("17:00:00", '%H:%M:%S').time())):
        return '6'
    if ((x >= datetime.strptime("17:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("19:00:00", '%H:%M:%S').time())):
        return '7'
    if ((x >= datetime.strptime("19:00:00", '%H:%M:%S').time()) and (x < datetime.strptime("21:00:00", '%H:%M:%S').time())):
        return '8'
    if ((x >= datetime.strptime("21:00:00", '%H:%M:%S').time()) and (x <= datetime.strptime("23:59:59", '%H:%M:%S').time())):
        return '9'

### Creating the list of all unique sensor IDs

In [73]:
sensors = pd.read_csv('map.txt')

In [74]:
sensors['Sensor'].unique()

array([ 214,  222,  256,  257,  258,  260,  261,  262,  264,  265,  266,
        268,  269,  270,  271,  272,  273,  274,  275,  276,  277,  279,
        280,  281,  282,  283,  285,  290,  291,  292,  293,  294,  295,
        296,  297,  298,  299,  303,  304,  307,  309,  310,  315,  318,
        319,  320,  321,  322,  323,  324,  326,  327,  328,  329,  330,
        331,  334,  336,  338,  339,  340,  341,  342,  343,  344,  345,
        346,  347,  348,  349,  350,  351,  352,  353,  354,  355,  356,
        357,  358,  359,  360,  361,  363,  365,  367,  368,  369,  370,
        371,  372,  373,  374,  375,  376,  377,  378,  379,  380,  383,
        384,  385,  386,  387,  388,  389,  390,  391,  392,  393,  394,
        395,  396,  397,  398,  399,  400,  401,  403,  404,  405,  406,
        407,  408,  410,  412,  413,  414,  415,  416,  417,  418,  419,
        420,  421,  422,  423,  424,  425,  426,  427,  428,  429,  430,
        432,  433,  434,  435,  436,  437,  438,  4

In [79]:
list1 = [ 214,  222,  256,  257,  258,  260,  261,  262,  264,  265,  266,
        268,  269,  270,  271,  272,  273,  274,  275,  276,  277,  279,
        280,  281,  282,  283,  285,  290,  291,  292,  293,  294,  295,
        296,  297,  298,  299,  303,  304,  307,  309,  310,  315,  318,
        319,  320,  321,  322,  323,  324,  326,  327,  328,  329,  330,
        331,  334,  336,  338,  339,  340,  341,  342,  343,  344,  345,
        346,  347,  348,  349,  350,  351,  352,  353,  354,  355,  356,
        357,  358,  359,  360,  361,  363,  365,  367,  368,  369,  370,
        371,  372,  373,  374,  375,  376,  377,  378,  379,  380,  383,
        384,  385,  386,  387,  388,  389,  390,  391,  392,  393,  394,
        395,  396,  397,  398,  399,  400,  401,  403,  404,  405,  406,
        407,  408,  410,  412,  413,  414,  415,  416,  417,  418,  419,
        420,  421,  422,  423,  424,  425,  426,  427,  428,  429,  430,
        432,  433,  434,  435,  436,  437,  438,  439,  440,  441,  442,
        444,  445,  446,  447,  448,  449,  450,  451,  452,  453,  454,
        455,  458,  459,  460,  461,  462,  464,  465,  466,  467,  468,
        469,  470,  471,  472,  473,  474,  475,  476,  477,  478,  479,
        480,  481,  482,  483,  485,  486,  487,  488,  489,  490,  491,
        492,  493,  494,  495,  496,  497,  498,  499,  500,  501,  502,
        503,  505]

### Concating the bits, segment and sensor ID columns in that order to get the words

In [86]:
def concatcol(x):
    return str(x[1])+str(x[2])+str(x[3])+str(x[4])+str(x[5])+str(x[8])+str(x[0])

### Creating per day per sensor file

In [71]:
def file_gen(x):
    global df5
    df5 = df2[df2['Date']==x]
    global df7
    df7 = pd.DataFrame(data={'SensorID':[0], 'bit1':[0], 'bit2':[0], 'bit3':[0], 'bit4':[0], 'bit5':[0], 'Starttime':[0], 'Endtime':[0]})
    df7.drop([0],axis=0, inplace=True)
    timeiter.apply(wordgen, axis=1)[:1]
    df7['Segment'] = df7['Starttime'].apply(get_segment)
    df7['Word'] = df7.apply(concatcol, axis=1)
    for y in list1:
        df5 = df7[df7['SensorID']==y]
        if(df5.shape[0]>0):
            df5.to_csv('Per Day Per Sensor/'+x+' '+str(y)+'.txt')

### A separate template of dates in the observation period

In [92]:
dates = pd.read_excel('1monthdates.xlsx', dtype=object)

In [93]:
dates.head()

,Dates,String
0,2006-06-19,19-06-06
1,2006-06-20,20-06-06
2,2006-06-21,21-06-06
3,2006-06-22,22-06-06
4,2006-06-23,23-06-06


In [ ]:
dates['String'].apply(file_gen) #actual file generation